In a simple RAG setup, we follow these steps:
1. **Data Ingestion**: Load and preprocess text data
2. **Chunking**: Break data into smaller chunks to improve retrieval performance
3. **Embedding**: Convert the text chunks into numerical representations using an emdedding model
4. **Semantic Search**: Retrieve relevant chunks based on user query
5. **Response Generation**: Use a language model to generate a response based on retrieval text

### 1. Set up environment

In [1]:
import fitz
import os
from dotenv import load_dotenv
import numpy as np
import json
from sklearn.metrics.pairwise import cosine_similarity
from openai import OpenAI

In [2]:
load_dotenv("../conf.env")

True

### 2. Ingest data(PDF for this example)

In [3]:
def extract_text_from_pdf(pdf_path):
    """
    Extracts text from a PDF file and prints the first `num_chars` characters.

    Args:
    pdf_path (str): Path to the PDF file.

    Returns:
    str: Extracted text from the PDF.
    """
    # Open the PDF file
    mypdf = fitz.open(pdf_path)
    all_text = ""  # Initialize an empty string to store the extracted text

    # Iterate through each page in the PDF
    for page in mypdf:
        all_text += page.get_text("text") + " "

    return all_text.strip()  # Return the extracted text

### 3. Chunk the extracted text

In [4]:
def chunk_text(text, n, overlap):
    """
    Chunks the given text into segments of n characters with overlap.

    Args:
    text (str): The text to be chunked.
    n (int): The number of characters in each chunk.
    overlap (int): The number of overlapping characters between chunks.

    Returns:
    List[str]: A list of text chunks.
    """
    chunks = []  # Initialize an empty list to store the chunks
    
    # Loop through the text with a step size of (n - overlap)
    for i in range(0, len(text), n - overlap):
        # Append a chunk of text from index i to i + n to the chunks list
        chunks.append(text[i:i + n])

    return chunks  # Return the list of text chunks # list[text]

#### Set up OpenAI Client
Access [this](https://aistudio.google.com/apikey) to get your free API key from Google Gemini 

In [5]:

client = OpenAI(
    api_key=os.getenv("GEMINI_API_KEY"),
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

now load pdf, extract text and split it into chunks

In [6]:
# Define the path to the PDF file
pdf_path = "../data/AI_Information.pdf"

# Extract text from the PDF file
extracted_text = extract_text_from_pdf(pdf_path)

# Chunk the extracted text into segments of 1000 characters with an overlap of 200 characters
text_chunks = chunk_text(extracted_text, 1000, 200)

# Print the number of text chunks created
print("Number of text chunks:", len(text_chunks))

# Print the first text chunk
print("\nFirst text chunk:")
print(text_chunks[0])

Number of text chunks: 42

First text chunk:
Understanding Artificial Intelligence 
Chapter 1: Introduction to Artificial Intelligence 
Artificial intelligence (AI) refers to the ability of a digital computer or computer-controlled robot 
to perform tasks commonly associated with intelligent beings. The term is frequently applied to 
the project of developing systems endowed with the intellectual processes characteristic of 
humans, such as the ability to reason, discover meaning, generalize, or learn from past 
experience. Over the past few decades, advancements in computing power and data availability 
have significantly accelerated the development and deployment of AI. 
Historical Context 
The idea of artificial intelligence has existed for centuries, often depicted in myths and fiction. 
However, the formal field of AI research began in the mid-20th century. The Dartmouth Workshop 
in 1956 is widely considered the birthplace of AI. Early AI research focused on problem-solving 
and 

### 4. Emdedding
Convert text into numerical vectors, which allow for efficient similarity search

In [7]:
def create_embeddings(text, model="gemini-embedding-001"):
    """
    Creates embeddings for the given text using the specified OpenAI model.

    Args:
    text (str): The input text for which embeddings are to be created.
    model (str): The model to be used for creating embeddings. Default is "gemini-embedding-001".

    Returns:
    dict: The response from the OpenAI API containing the embeddings.
    """
    # Create embeddings for the input text using the specified model
    response = client.embeddings.create(
        model=model,
        input=text
    )

    return response  # Return the response containing the embeddings # CreateEmbeddingResponse

# Create embeddings for the text chunks
response = create_embeddings(text_chunks) # :CreateEmbeddingResponse

In [8]:
response.data # :list[Embedding]
response.data[1].embedding # :(x,1)
np.array(response.data[0].embedding) # :NDArray(x,)

array([ 0.0049712 ,  0.00908941,  0.02203033, ...,  0.00198734,
       -0.00881044, -0.01199376], shape=(3072,))

In [9]:
[np.array(response.data[i].embedding) for i in range(len(response.data))] # list[NDA(x,)]

[array([ 0.0049712 ,  0.00908941,  0.02203033, ...,  0.00198734,
        -0.00881044, -0.01199376], shape=(3072,)),
 array([-0.00183863, -0.01369122,  0.02448579, ...,  0.00070203,
        -0.00132361, -0.00723084], shape=(3072,)),
 array([-0.01276625, -0.0080712 , -0.01626536, ..., -0.0035511 ,
         0.01299587,  0.00095135], shape=(3072,)),
 array([-0.01317397, -0.00019218, -0.00754517, ..., -0.00856328,
        -0.00673063, -0.01202932], shape=(3072,)),
 array([ 0.01870531,  0.01208002,  0.00602027, ..., -0.00929642,
         0.0001431 , -0.00295126], shape=(3072,)),
 array([ 0.01819538, -0.00470313,  0.01499009, ...,  0.01054152,
        -0.00521335, -0.01171913], shape=(3072,)),
 array([-0.01056309, -0.0066891 ,  0.01673094, ..., -0.01151042,
         0.00066451,  0.0060216 ], shape=(3072,)),
 array([-0.02305629,  0.00502344,  0.00977692, ..., -0.02509176,
         0.00374365,  0.00935353], shape=(3072,)),
 array([-0.0233781 , -0.01911625, -0.00713572, ..., -0.00426102,
       

In [10]:
embeddings_matrix = [np.array(response.data[i].embedding) for i in range(len(response.data))] # :list(np.array)
similarity_matrix = cosine_similarity(embeddings_matrix)

for i, text1 in enumerate(text_chunks):
    for j in range(i + 1, 5):
        text2 = text_chunks[j]
        similarity = similarity_matrix[i, j]
        print(f"Similarity between '{i}' and '{j}': {similarity:.4f}")

Similarity between '0' and '1': 0.8222
Similarity between '0' and '2': 0.7588
Similarity between '0' and '3': 0.6605
Similarity between '0' and '4': 0.6777
Similarity between '1' and '2': 0.8194
Similarity between '1' and '3': 0.6857
Similarity between '1' and '4': 0.7175
Similarity between '2' and '3': 0.7799
Similarity between '2' and '4': 0.7008
Similarity between '3' and '4': 0.7937


implement semantic search to find contextual similarity

In [11]:
def semantic_search(query, text_chunks, embeddings, k=5):
    """
    Performs semantic search on the text chunks using the given query and embeddings.

    Args:
    query (str): The query for the semantic search.
    text_chunks (List[str]): A list of text chunks to search through.
    embeddings (List[dict]): A list of embeddings for the text chunks.
    k (int): The number of top relevant text chunks to return. Default is 5.

    Returns:
    List[str]: A list of the top k most relevant text chunks based on the query.
    """
    # Create an embedding for the query
    query_embedding = create_embeddings(query).data[0].embedding # (x,1)
    similarity_scores = []  # Initialize a list to store similarity scores

    # Calculate similarity scores between the query embedding and each text chunk embedding
    for i, chunk_embedding in enumerate(embeddings):
        similarity_score = cosine_similarity(np.array(query_embedding).reshape(1, -1), np.array(chunk_embedding.embedding).reshape(1, -1))
                        #cos_sim(NDA(1,x), NDA(x,).reshape(1, -1) => NDA(1,x))
        similarity_scores.append((i, similarity_score))  # Append the index and similarity score

    # Sort the similarity scores in descending order
    similarity_scores.sort(key=lambda x: x[1], reverse=True)
    # Get the indices of the top k most similar text chunks
    top_indices = [index for index, _ in similarity_scores[:k]]
    # Return the top k most relevant text chunks
    return [text_chunks[index] for index in top_indices]

running a query on extracted chunks

In [12]:
# Load the validation data from a JSON file
with open('../data/val.json') as f:
    data = json.load(f)

# Extract the first query from the validation data
query = data[0]['question']

# Perform semantic search to find the top 2 most relevant text chunks for the query
top_chunks = semantic_search(query, text_chunks, response.data, k=2)

# Print the query
print("Query:", query)

# Print the top 2 most relevant text chunks
for i, chunk in enumerate(top_chunks):
    print(f"Context {i + 1}:\n{chunk}\n=====================================")


Query: What is 'Explainable AI' and why is it considered important?
Context 1:
lainability 
Many AI systems, particularly deep learning models, are "black boxes," making it difficult to 
understand how they arrive at their decisions. Enhancing transparency and explainability is 
crucial for building trust and accountability. 
 
  
Privacy and Security 
AI systems often rely on large amounts of data, raising concerns about privacy and data security. 
Protecting sensitive information and ensuring responsible data handling are essential. 
Job Displacement 
The automation capabilities of AI have raised concerns about job displacement, particularly in 
industries with repetitive or routine tasks. Addressing the potential economic and social impacts 
of AI-driven automation is a key challenge. 
Autonomy and Control 
As AI systems become more autonomous, questions arise about control, accountability, and the 
potential for unintended consequences. Establishing clear guidelines and ethical fra

In [13]:
np.array(create_embeddings(query).data[0].embedding)

array([-0.02039474,  0.00263954,  0.009839  , ..., -0.02887365,
        0.00846821,  0.01495596], shape=(3072,))

### 5. Generate response

In [14]:
# Define the system prompt for the AI assistant
system_prompt = "You are an AI assistant that strictly answers based on the given context. If the answer cannot be derived directly from the provided context, respond with: 'I do not have enough information to answer that.'"

def generate_response(system_prompt, user_message, model="gemini-2.5-flash"):
    """
    Generates a response from the AI model based on the system prompt and user message.

    Args:
    system_prompt (str): The system prompt to guide the AI's behavior.
    user_message (str): The user's message or query.
    model (str): The model to be used for generating the response. Default is "meta-llama/Llama-2-7B-chat-hf".

    Returns:
    dict: The response from the AI model.
    """
    response = client.chat.completions.create(
        model=model,
        temperature=0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_message}
        ]
    )
    return response

# Create the user prompt based on the top chunks
user_prompt = "\n".join([f"Context {i + 1}:\n{chunk}\n=====================================\n" for i, chunk in enumerate(top_chunks)])
user_prompt = f"{user_prompt}\nQuestion: {query}"

# Generate AI response
ai_response = generate_response(system_prompt, user_prompt)

In [15]:
ai_response_text = ai_response.choices[0].message.content

In [16]:
print(ai_response)

ChatCompletion(id='ZqeAaPeWBIDUz7IP9sr16Aw', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Explainable AI (or explainability) refers to making AI systems understandable by providing insights into their decision-making processes, especially for "black box" models like deep learning. It is considered important because enhancing transparency and explainability is crucial for building trust and accountability, and it helps users assess the reliability and fairness of AI systems.', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None))], created=1753261926, model='gemini-2.5-flash', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=66, prompt_tokens=471, total_tokens=1019, completion_tokens_details=None, prompt_tokens_details=None))


### Evaluate

In [17]:
# Define the system prompt for the evaluation system
evaluate_system_prompt = "You are an intelligent evaluation system tasked with assessing the AI assistant's responses. If the AI assistant's response is very close to the true response, assign a score of 1. If the response is incorrect or unsatisfactory in relation to the true response, assign a score of 0. If the response is partially aligned with the true response, assign a score of 0.5."

# Create the evaluation prompt by combining the user query, AI response, true response, and evaluation system prompt
evaluation_prompt = f"User Query: {query}\nAI Response:\n{ai_response.choices[0].message.content}\nTrue Response: {data[0]['ideal_answer']}\n{evaluate_system_prompt}"

# Generate the evaluation response using the evaluation system prompt and evaluation prompt
evaluation_response = generate_response(evaluate_system_prompt, evaluation_prompt)

# Print the evaluation response
print(evaluation_response.choices[0].message.content)

Score: 1


In [18]:
np.array(create_embeddings(ai_response.choices[0].message.content).data[0].embedding)

array([ 0.00397952,  0.01158977,  0.0064257 , ..., -0.02235902,
        0.02382587, -0.00643594], shape=(3072,))

In [19]:
np.array(create_embeddings(data[0]['ideal_answer']).data[0].embedding)

array([-0.00634933,  0.01132035,  0.00591608, ..., -0.02739728,
        0.02021963, -0.00203079], shape=(3072,))

In [20]:
ai_gen = np.array(create_embeddings(ai_response_text).data[0].embedding).reshape(1, -1) # (x,).reshape(1, -1) => (1,x)
ideal_re = np.array(create_embeddings(data[0]['ideal_answer']).data[0].embedding).reshape(1, -1) # (x,).reshape(1, -1) => (1,x)
print(ai_gen.shape, ideal_re.shape)

cosine_similarity(ai_gen, ideal_re)

(1, 3072) (1, 3072)


array([[0.93485396]])